# First part

In [1]:
!pip install bs4



  Stored in directory: /home/dsxuser/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
import requests
import pandas as pd
import bs4
from bs4 import BeautifulSoup

In [3]:
url=r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


I will use Beautiful Soup to scrape the table

In [4]:
toronto_resp=requests.get(url).text
soup=BeautifulSoup(toronto_resp,'lxml')
#soup.prettify()

In [5]:
#find first table on page

toronto_table = soup.find_all('table')[0]


In [35]:
for tr in toronto_table.find_all('tr')[1:]:
    
    tds = tr.find_all('td')
    
    print("Postal Code: %s, Borough: %s, Neighborhood: %s" % (tds[0].text.strip(), tds[1].text.strip(), tds[2].text.strip()))
    
    

Postal Code: M1A, Borough: Not assigned, Neighborhood: Not assigned
Postal Code: M2A, Borough: Not assigned, Neighborhood: Not assigned
Postal Code: M3A, Borough: North York, Neighborhood: Parkwoods
Postal Code: M4A, Borough: North York, Neighborhood: Victoria Village
Postal Code: M5A, Borough: Downtown Toronto, Neighborhood: Regent Park, Harbourfront
Postal Code: M6A, Borough: North York, Neighborhood: Lawrence Manor, Lawrence Heights
Postal Code: M7A, Borough: Downtown Toronto, Neighborhood: Queen's Park, Ontario Provincial Government
Postal Code: M8A, Borough: Not assigned, Neighborhood: Not assigned
Postal Code: M9A, Borough: Etobicoke, Neighborhood: Islington Avenue, Humber Valley Village
Postal Code: M1B, Borough: Scarborough, Neighborhood: Malvern, Rouge
Postal Code: M2B, Borough: Not assigned, Neighborhood: Not assigned
Postal Code: M3B, Borough: North York, Neighborhood: Don Mills
Postal Code: M4B, Borough: East York, Neighborhood: Parkview Hill, Woodbine Gardens
Postal Code: 

Next I populate the dataframe df with 'PostalCode','Borough','Neighborhood' from the scraped table. Removal of non assigned boroughs and replacement of not assigned neighborhoods with borough is already done within the loop that populates the df.

In [36]:
#toronto_table
df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

i=0
for tr in toronto_table.find_all('tr')[1:]:
    
    tds = tr.find_all('td')

    
    if tds[1].text.strip()=='Not assigned':#if borough not assigned skip row
        
         pass

    else:
        
        df.loc[i,'PostalCode']=tds[0].text.strip()
        df.loc[i,'Borough']=tds[1].text.strip()

        if tds[2].text.strip()=='Not assigned': #if neighborhood not assigned
            
            df.loc[i,'Neighborhood']=tds[1].text.strip() #then neighborhood same as borough
        else:
            df.loc[i,'Neighborhood']=tds[2].text.strip()

        i=i+1

df.head()
    
    

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Getshape of DF:

In [40]:

#pbrows =[ i for i in df.loc[df['Neighborhood']=='Not assigned'].index]
#df.loc[pbrows,'Neighborhood']=df.loc[pbrows,'Borough']


df.shape

(103, 3)

# This ends the first part of the assigment.

# Second Part 

In [41]:
!pip install geopy

In [42]:
!pip install folium

     |████████████████████████████████| 102kB 9.4MB/s ta 0:00:011


In [45]:
!pip install geocoder


     |████████████████████████████████| 102kB 7.5MB/s ta 0:00:011


In [46]:
import geocoder
import numpy as np

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Since I had problems with the google api I used the csv to get lat, lng for each Postal Code.

In [70]:
#for index, row in df.iterrows():
    
   # trials=0
    
    #lat_lng_coords = None
    
    #print(index, row['PostalCode'])
    
    # loop until you get the coordinates
   # while(trials <10):
         # g = geocoder.google('{}, Toronto, Ontario'.format(row['PostalCode']))
         # lat_lng_coords = g.latlng
        #  trials=trials+1
         # if lat_lng_coords is not None:
         #   break
          #  print(lat_lng_coords[0])

    #df.loc[index,'Latitude'] = lat_lng_coords[0]
    #df.loc[index,'Longitude'] = lat_lng_coords[1]
        

In [61]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')


Data downloaded!


In [62]:
with open('Geospatial_Coordinates.csv') as latng:
    df_latlng = pd.read_csv(latng)

In [63]:
df_latlng.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I create a loop to map each postal code in the dataframe with the neighborhoods to each postalcode in the new df with the position data:

In [108]:
df2=pd.merge(df, df_latlng, left_on='PostalCode',right_on='Postal Code',
      how='left', sort=False)

df2=df2.drop(['Postal Code'], axis=1)


df2.head()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Here is the final dataframe:

In [110]:
#compare wuth assignment data

#df2[df2['PostalCode']=='M2H']
#df2.shape



(103, 5)

## This ends the second part of the assignment.

## Third Part

Create new df neighborhoods with only boroughs that contain the word Toronto:

In [111]:
neighborhoods=df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)

neighborhoods.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [112]:
neighborhoods.groupby('Borough').count()

,PostalCode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
West Toronto,6,6,6,6


In [113]:
neighborhoods.dtypes

neighborhoods.shape


(39, 5)

In [115]:

neighborhoods.dropna(subset=['Latitude'], axis=0, inplace=True)
neighborhoods.dropna(subset=['Longitude'], axis=0, inplace=True)


neighborhoods.reset_index(drop=True, inplace=True)

neighborhoods.shape



(39, 5)

In [116]:
print('The dataframe has {} boroughs and {} postal codes.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 39 postal codes.


Get position data for Toronto City in order to be able to center the folium map.

In [76]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [287]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Foursquare Input data:

In [78]:
CLIENT_ID = 'NROBF02A4FI4HHG101DGTBBKK5LGWK2BZWBNZMBXGLIGCH0W' # your Foursquare ID
CLIENT_SECRET = '0RKFO0HXIHCQENFQASBHKV5ZCF2TKXWW1V53GDFEMQ2A2KF5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


Limit=100
Radius=500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NROBF02A4FI4HHG101DGTBBKK5LGWK2BZWBNZMBXGLIGCH0W
CLIENT_SECRET:0RKFO0HXIHCQENFQASBHKV5ZCF2TKXWW1V53GDFEMQ2A2KF5


 # Analysis for one neighborhood

In [117]:
neighborhood_latitude = neighborhoods.iloc[0, 3] # neighborhood latitude value
neighborhood_longitude = neighborhoods.iloc[0, 4] # neighborhood longitude value

neighborhood_name = neighborhoods.iloc[0, 2] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                              neighborhood_latitude, 
                                                            neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [118]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    Radius, 
    Limit)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NROBF02A4FI4HHG101DGTBBKK5LGWK2BZWBNZMBXGLIGCH0W&client_secret=0RKFO0HXIHCQENFQASBHKV5ZCF2TKXWW1V53GDFEMQ2A2KF5&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [119]:
results = requests.get(url).json()
#results

In [120]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [121]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [122]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


## Analysis for all neighborhoods in Toronto

In [129]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
       
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        try:
             
        # make the GET request
            results0 = requests.get(url).json()
            results = results0['response']['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

        except :
                pass
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [185]:
# type your answer here

neighborhood_venues= getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



In [131]:
print(neighborhood_venues.shape)
neighborhood_venues.head()

(1358, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Check how many venues where retuned for each neighbor (I set the limit to 100)

In [132]:
neighborhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,64,64,64,64,64,64
Church and Wellesley,81,81,81,81,81,81
Davisville,35,35,35,35,35,35
"Dufferin, Dovercourt Village",16,16,16,16,16,16
"First Canadian Place, Underground city",100,100,100,100,100,100


In [186]:
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

There are 236 uniques categories.


In [187]:
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 


# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot=neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
neighborhood_onehot.shape

(1626, 236)

In [190]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.024691,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,...,0.012346,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [191]:
neighborhood_grouped.shape


#there only 33 neighborhoods instead of 39 left 

(39, 236)

In [192]:
temp =neighborhood_grouped[neighborhood_grouped['Neighborhood'] == 'Bay Street'].T.reset_index()

temp.head()
#temp.columns

,index
0,Neighborhood
1,Yoga Studio
2,Afghan Restaurant
3,Airport
4,Airport Food Court


In [193]:
num_top_venues = 5

for hood in neighborhood_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =neighborhood_grouped[neighborhood_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2            Beer Bar  0.03
3  Seafood Restaurant  0.03
4         Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
               venue  freq
0               Café  0.13
1     Breakfast Spot  0.09
2        Coffee Shop  0.09
3             Bakery  0.04
4  Convenience Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0           Yoga Studio  0.06
1               Brewery  0.06
2  Gym / Fitness Center  0.06
3         Garden Center  0.06
4                Garden  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4          Boutique  0.06


----Central Bay Street----
                 v

In [148]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Let's print each neighborhood along with the top 10 most common venues:

In [194]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Restaurant,Bakery,Café,Pub,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Gym,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Garden Center,Recording Studio,Brewery,Auto Workshop,Fast Food Restaurant,Farmers Market,Burrito Place,Garden,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Bar
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Department Store,Salad Place,Bubble Tea Shop,Burger Joint,Yoga Studio


In [195]:
# set number of clusters
kclusters = 8

neighborhoods_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)


neighborhoods_grouped_clustering

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.024691,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.00,...,0.012346,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [196]:

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 6, 1, 1, 7, 1, 1, 1, 6], dtype=int32)

In [275]:

# run dbscan-means clustering
from sklearn.cluster import DBSCAN 
import matplotlib.pyplot as plt 
%matplotlib inline

epsilon = 0.3
minimumSamples = 4
db = DBSCAN(eps=epsilon, min_samples=minimumSamples).fit(neighborhoods_grouped_clustering)
dbscan_labels= db.labels_


In [276]:

#Lets Replace all elements with 'True' in core_samples_mask that are in the cluster, 'False' if the points are outliers.

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
core_samples_mask


array([ True,  True,  True, False,  True, False,  True,  True,  True,
       False,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True])

In [277]:

dbscan_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
dbscan_unique_labels = set(dbscan_labels)
dbscan_unique_labels

{-1, 0}

In [280]:
#neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop(['Cluster Labels', 'Cluster Labels DBSCAN'], axis=1)

In [281]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels DBSCAN', db.labels_)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted.head()

,Cluster Labels,Cluster Labels DBSCAN,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Restaurant,Bakery,Café,Pub,Irish Pub
1,1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Gym,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,6,0,"Business reply mail Processing Centre, South C...",Yoga Studio,Garden Center,Recording Studio,Brewery,Auto Workshop,Fast Food Restaurant,Farmers Market,Burrito Place,Garden,Restaurant
3,1,-1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Bar
4,1,0,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Department Store,Salad Place,Bubble Tea Shop,Burger Joint,Yoga Studio


In [282]:
neighborhoods_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhoods_merged.head() # check the first columns

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Cluster Labels DBSCAN,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Yoga Studio,Cosmetics Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,0,Coffee Shop,Sushi Restaurant,Gym,Distribution Center,Restaurant,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Fast Food Restaurant,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Restaurant,Art Gallery,Italian Restaurant,Lingerie Store,Department Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,-1,Health Food Store,Trail,Pub,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant


In [283]:

neighborhoods_merged_final=neighborhoods_merged

# simply drop whole row with NaN in "Cluster" column
#neighborhoods_merged_final.dropna(subset=['Cluster Labels'], axis=0, inplace=True)

# reset index, because we droped two rows
#neighborhoods_merged_final.reset_index(drop=True, inplace=True)

neighborhoods_merged_final['Cluster Labels'] = neighborhoods_merged_final['Cluster Labels'].astype('int')

neighborhoods_merged_final['Cluster Labels DBSCAN'] = neighborhoods_merged_final['Cluster Labels DBSCAN'].astype('int')


In [284]:
neighborhoods_merged_final.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Cluster Labels DBSCAN,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Yoga Studio,Cosmetics Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,0,Coffee Shop,Sushi Restaurant,Gym,Distribution Center,Restaurant,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Fast Food Restaurant,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Restaurant,Art Gallery,Italian Restaurant,Lingerie Store,Department Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,-1,Health Food Store,Trail,Pub,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant


Create Folium map with K-means clusters:

In [285]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [cm.colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged_final['Latitude'], neighborhoods_merged_final['Longitude'], neighborhoods_merged_final['Neighborhood'], neighborhoods_merged_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Create Folium map with DB-scan clusters:

In [286]:
# create map for dbscan clusters
dbscan_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
colors_array = plt.cm.Spectral(np.linspace(0, 1, len(dbscan_unique_labels)))

dbscancolors = [cm.colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster  in zip(neighborhoods_merged_final['Latitude'], neighborhoods_merged_final['Longitude'], neighborhoods_merged_final['Neighborhood'], neighborhoods_merged_final['Cluster Labels DBSCAN']):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
            
   #cluster==-1 are outliers
    
    col = dbscancolors[cluster]
      
    
    folium.CircleMarker(
        
        [lat, lon],
        radius=5,
        popup=label,
        color= col,
        
        fill=True,
        
        fill_color=col,
        
        fill_opacity=0.7
    
    ).add_to(dbscan_map_clusters)
       
dbscan_map_clusters

DB Scan basically yields similar results than K-means clustering. It just identifies all points that are not in the cluster with the biggest number of constituents as outliers. K-means is not able to identify outliers and thus puts these points into separate clusters.


Let's see if we can characterize the 8 clusters identified by the k-means method further by their attributes:


In [204]:
neighborhoods_merged_final.loc[neighborhoods_merged_final['Cluster Labels'] == 0, neighborhoods_merged_final.columns[[2] + list(range(5, neighborhoods_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,0,Health Food Store,Trail,Pub,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant


In [205]:
neighborhoods_merged_final.loc[neighborhoods_merged_final['Cluster Labels'] == 1, neighborhoods_merged_final.columns[[2] + list(range(5, neighborhoods_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Yoga Studio,Cosmetics Shop
1,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Sushi Restaurant,Gym,Distribution Center,Restaurant,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
2,"Garden District, Ryerson",1,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Fast Food Restaurant,Ramen Restaurant
3,St. James Town,1,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Restaurant,Art Gallery,Italian Restaurant,Lingerie Store,Department Store
5,Berczy Park,1,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Restaurant,Bakery,Café,Pub,Irish Pub
6,Central Bay Street,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Department Store,Salad Place,Bubble Tea Shop,Burger Joint,Yoga Studio
8,"Richmond, Adelaide, King",1,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Clothing Store,Deli / Bodega,Bookstore,American Restaurant
10,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Aquarium,Hotel,Café,Brewery,Sporting Goods Shop,Restaurant,Italian Restaurant,Fried Chicken Joint,Scenic Lookout
11,"Little Portugal, Trinity",1,Bar,Asian Restaurant,Coffee Shop,Restaurant,Café,Men's Store,Vegetarian / Vegan Restaurant,Ice Cream Shop,Italian Restaurant,Gift Shop
12,"The Danforth West, Riverdale",1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Yoga Studio,Pub,Indian Restaurant,Bookstore


In [209]:
neighborhoods_merged_final.loc[neighborhoods_merged_final['Cluster Labels'] == 2, neighborhoods_merged_final.columns[[2] + list(range(5, neighborhoods_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,2,Park,Bus Line,Swim School,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [210]:
neighborhoods_merged_final.loc[neighborhoods_merged_final['Cluster Labels'] == 3, neighborhoods_merged_final.columns[[2] + list(range(5, neighborhoods_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",3,Park,Trail,Restaurant,Comfort Food Restaurant,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Rosedale,3,Park,Playground,Trail,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [211]:
neighborhoods_merged.loc[neighborhoods_merged_final['Cluster Labels'] == 4,neighborhoods_merged_final.columns[[2] + list(range(5, neighborhoods_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,4,Home Service,Garden,Women's Store,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [212]:
neighborhoods_merged.loc[neighborhoods_merged_final['Cluster Labels'] == 5,neighborhoods_merged_final.columns[[2] + list(range(5, neighborhoods_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Forest Hill North & West, Forest Hill Road Park",5,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [215]:
neighborhoods_merged.loc[neighborhoods_merged_final['Cluster Labels'] == 6,neighborhoods_merged_final.columns[[2] + list(range(5, neighborhoods_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,"India Bazaar, The Beaches West",6,Park,Fast Food Restaurant,Gym,Pub,Liquor Store,Sandwich Place,Burrito Place,Italian Restaurant,Restaurant,Fish & Chips Shop
20,Davisville North,6,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Department Store,Gym / Fitness Center,Pizza Place,Park,Electronics Store,Eastern European Restaurant
38,"Business reply mail Processing Centre, South C...",6,Yoga Studio,Garden Center,Recording Studio,Brewery,Auto Workshop,Fast Food Restaurant,Farmers Market,Burrito Place,Garden,Restaurant


In [216]:
neighborhoods_merged.loc[neighborhoods_merged_final['Cluster Labels'] == 7,neighborhoods_merged_final.columns[[2] + list(range(5, neighborhoods_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Christie,7,Grocery Store,Café,Park,Coffee Shop,Baby Store,Diner,Italian Restaurant,Restaurant,Nightclub,Candy Store
9,"Dufferin, Dovercourt Village",7,Pharmacy,Bakery,Middle Eastern Restaurant,Bar,Grocery Store,Park,Bank,Supermarket,Pizza Place,Music Venue
22,"High Park, The Junction South",7,Mexican Restaurant,Park,Thai Restaurant,Café,Fast Food Restaurant,Arts & Crafts Store,Bar,Flea Market,Bakery,Speakeasy


It is rather difficult to give a meaning to the different clusters. Cluster 3 seems to be a green area with many parks.  Cluster 0 is on the Beach. Cluster 1 with the most of neighborhoods may be characterized by a lot of gastronomy (cafes, bars, restaurants) and is rather located in the center of Toronto. Cluster 5 has a lot of Jewelry stores.  Cluster 7 has a lot of grocery stores.